# Simulación de recréditos inhabilitados con margen a 12 meses

Este cuaderno hace una simulación del otorgamiento de recrédito a clientes inhabilitados buscando un margen financiero a 12 meses.

## Funciones útiles internas

In [1]:
import pandas as pd
import warnings
import math
warnings.filterwarnings("ignore")
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pyodbc
import sys
sys.path.append(r"C:\Users\mariajose_chinchilla\Documents\GitHub\proyectos_varios\codigos") #aca pongan la ruta donde lo gaurden

from utils import ingresos_financieros, ingreso_fin_entre_fechas, meses_entre_fechas, calcular_cuota

In [2]:
# modelar cvf
def factor_bienestar(monto):
    if monto >= 120000:
        factor = 9
    elif monto <= 120000:
        factor = 9.5
    return factor

def cvf(mf_cancelado: float, factor_bienestar: float, desembolso_neto: float, 
        reserva_cancelada: float, reserva_nueva: float, desembolso_bruto: float, alpha: float) -> float:
        tasa = (mf_cancelado + factor_bienestar * desembolso_neto + reserva_nueva - reserva_cancelada) / desembolso_bruto + 100 * alpha
        return min(max(19.5, tasa), 25)
        #return tasa

def porcentaje_comision_desembolso(x):
    if x < 125000:
        porcentaje = 0.05
    elif x >= 125000:
        porcentaje = 0.02
    return porcentaje

dic_pdi = {"A": 0.036, "B": 0.086, "C": 0.156, "D": 0.325, "E": 1}
dic_pi = {"A": 0.45, "B": 0.45, "C": 0.45, "D": 0.45, "E": 1}

def reservas_constituidas(monto, meses_transcurridos, categoria):
    pdi = dic_pdi.get(categoria)    
    pi = dic_pi.get(categoria)
    reservas = monto * pdi * pi
    return reservas

## Programación de escenario

In [3]:
no_mes_simulacion = 9
nombre_mes_simulacion = "SEPTIEMBRE"

In [4]:
cartera = pd.read_excel("../db/datos.xlsx")
cartera["TASA"] = cartera["TASA"] / 1200
cartera = cartera[cartera["TASA"] > 0]

In [5]:
cartera.columns

Index(['FEC_SALDO', 'NO_CREDITO', 'CODIGO_CLIENTE', 'F_APERTURA',
       'FECHA_DESEMBOLSO', 'TIPO_PAGO', 'CATEGORIA_INTERNA', 'TASA', 'PLAZO',
       'FACTOR_TASA', 'SALDO', 'MONTO_DESEMBOLSADO', 'DESEMBOLSO_NETO'],
      dtype='object')

In [6]:
# filtros para seleccionar clientes que nos interesan
mascara_remesa = cartera["TIPO_PAGO"] == "REMESA"
cartera = cartera.loc[mascara_remesa]

In [7]:
# agrupar por cliente para sacar qué clientes están inhabilitados
import numpy as np
cartera["CREDITO_CUMPLE_15"] = np.where(cartera["SALDO"] / cartera["MONTO_DESEMBOLSADO"] > 0.85, 0, 1)
cartera_agrupado = cartera.groupby(by="CODIGO_CLIENTE")["CREDITO_CUMPLE_15"].min().reset_index()
cartera_agrupado.columns = ["CODIGO_CLIENTE", "INCUMPLE"]
cartera_agrupado["PAGA RESERVAS"] = 1 - cartera_agrupado["INCUMPLE"]
cartera = pd.merge(cartera, cartera_agrupado, how="inner", on="CODIGO_CLIENTE")
cartera.drop("INCUMPLE", inplace=True, axis=1)

In [8]:
mascara_incumplimiento = cartera["PAGA RESERVAS"] == 1
incumplen = cartera.loc[mascara_incumplimiento]
incumplen.head(2)

,FEC_SALDO,NO_CREDITO,CODIGO_CLIENTE,F_APERTURA,FECHA_DESEMBOLSO,TIPO_PAGO,CATEGORIA_INTERNA,TASA,PLAZO,FACTOR_TASA,SALDO,MONTO_DESEMBOLSADO,DESEMBOLSO_NETO,CREDITO_CUMPLE_15,PAGA RESERVAS
0,45535,12408863300,100116409,45535,45535.0,REMESA,A,0.013333,120,312000.00,19500.00,19500.0,19500.0,0,1
1,45535,12405496084,100116409,45435,45435.0,REMESA,A,0.012500,120,967870.35,64524.69,65000.0,65000.0,0,1


In [9]:
incumplen[incumplen["CODIGO_CLIENTE"] ==  100588200]

,FEC_SALDO,NO_CREDITO,CODIGO_CLIENTE,F_APERTURA,FECHA_DESEMBOLSO,TIPO_PAGO,CATEGORIA_INTERNA,TASA,PLAZO,FACTOR_TASA,SALDO,MONTO_DESEMBOLSADO,DESEMBOLSO_NETO,CREDITO_CUMPLE_15,PAGA RESERVAS
105721,45535,12310386080,100588200,45227,45227.0,REMESA,A,0.011500,120,146211.000,10595.00,11000.0,11000.0,0,1
105722,45535,12208101046,100588200,44797,44797.0,REMESA,A,0.012667,108,1316173.928,86590.39,99200.0,99200.0,0,1


In [11]:
# calcular ingresos que se cancelarian, pasivos y comisiones
incumplen["F_APERTURA"] = pd.to_datetime(incumplen["F_APERTURA"], format="%d/%m/%Y")
incumplen["MESES TRANSCURRIDOS"] = incumplen.apply(lambda x: meses_entre_fechas(x["F_APERTURA"], datetime.today()), axis=1)
incumplen["INGRESOS DEL CREDITO A 12 MESES"] = incumplen.apply(lambda x: ingreso_fin_entre_fechas(x["MONTO_DESEMBOLSADO"], x["TASA"], x["PLAZO"], x["MESES TRANSCURRIDOS"], x["MESES TRANSCURRIDOS"] + 12), axis=1)
alpha = (0.08 + 0.0025) / (1 - 0.146)
incumplen["PASIVOS DEL CREDITO A 12 MESES"] = incumplen.apply(lambda x: ingreso_fin_entre_fechas(x["MONTO_DESEMBOLSADO"], alpha/12, x["PLAZO"], x["MESES TRANSCURRIDOS"], x["MESES TRANSCURRIDOS"] + 12), axis=1)
incumplen["RESERVAS DEL CREDITO"] = incumplen.apply(lambda x: reservas_constituidas(x["MONTO_DESEMBOLSADO"], x["MESES TRANSCURRIDOS"], x["CATEGORIA_INTERNA"]), axis=1)
incumplen["PORCENTAJE COMISION"] = incumplen.apply(lambda x: porcentaje_comision_desembolso(x["DESEMBOLSO_NETO"]), axis=1)
incumplen["COMISIONES QUE SE TRAERIAN SI SE HACE RECREDITO"] = incumplen.apply(lambda x: ingreso_fin_entre_fechas(x["DESEMBOLSO_NETO"], x["PORCENTAJE COMISION"]/12, x["PLAZO"], x["MESES TRANSCURRIDOS"], x["PLAZO"]), axis=1)

incumplen["COMISIONES CREDITO"] = incumplen.apply(lambda x: ingreso_fin_entre_fechas(x["MONTO_DESEMBOLSADO"], x["PORCENTAJE COMISION"]/12, x["PLAZO"], x["MESES TRANSCURRIDOS"], x["MESES TRANSCURRIDOS"] + 12), axis=1)
#incumplen["COMISIONES FIN DE AÑO"] = incumplen.apply(lambda x: ingreso_fin_entre_fechas(x["MONTO_DESEMBOLSADO"], x["TASA"], x["PLAZO"], x["MESES TRANSCURRIDOS"], x["MESES TRANSCURRIDOS"] + 12 - no_mes_simulacion), axis=1)

# calcular columnas de agregacion importantes
incumplen_agrupado = incumplen.groupby(by="CODIGO_CLIENTE").agg(
                                        DEUDA_INTERNA=("SALDO", "sum"),
                                        INGRESOS_A_CANCELAR_12_MESES=("INGRESOS DEL CREDITO A 12 MESES", "sum"),
                                        PASIVOS_A_CANCELAR=("PASIVOS DEL CREDITO A 12 MESES", "sum"),
                                        RESERVA_CONSTITUIDA=("RESERVAS DEL CREDITO", "sum"),
                                        COMISIONES_12_MESES_ACTIVOS=("COMISIONES CREDITO", "sum"),
                                        COMISIONES_A_TRAERSE=("COMISIONES QUE SE TRAERIAN SI SE HACE RECREDITO", "sum")
)

incumplen = pd.merge(incumplen, incumplen_agrupado, how="inner", on="CODIGO_CLIENTE")

# Sacar mf cancelado
incumplen["MF CANCELADO CLIENTE"] = incumplen["INGRESOS DEL CREDITO A 12 MESES"] - incumplen["PASIVOS_A_CANCELAR"] - incumplen["RESERVA_CONSTITUIDA"] + incumplen["COMISIONES_12_MESES_ACTIVOS"] 

# ahora la difernecia de margenes pero a fin de año
incumplen["INGRESOS CREDITO FIN DE AÑO"] = incumplen.apply(lambda x: ingreso_fin_entre_fechas(x["MONTO_DESEMBOLSADO"], x["TASA"], x["PLAZO"], x["MESES TRANSCURRIDOS"], x["MESES TRANSCURRIDOS"] + 12 - no_mes_simulacion), axis=1)
incumplen["RESERVAS A CANCELAR FIN DE AÑO"] = incumplen.apply(lambda x: reservas_constituidas(x["MONTO_DESEMBOLSADO"], x["MESES TRANSCURRIDOS"] + 12 - no_mes_simulacion, x["CATEGORIA_INTERNA"]), axis=1)
incumplen["PASIVOS A CANCELAR FIN DE AÑO"] = incumplen.apply(lambda x: ingreso_fin_entre_fechas(x["MONTO_DESEMBOLSADO"], x["TASA"], x["PLAZO"], x["MESES TRANSCURRIDOS"], x["MESES TRANSCURRIDOS"] + 12 - no_mes_simulacion), axis=1)
incumplen["MF A CANCELAR FIN DE AÑO"] = incumplen["INGRESOS CREDITO FIN DE AÑO"] - incumplen["RESERVAS A CANCELAR FIN DE AÑO"] - incumplen["PASIVOS A CANCELAR FIN DE AÑO"] 

ValueError: time data "45535" doesn't match format "%d/%m/%Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
incumplen["F_APERTURA"]

In [9]:
incumplen_temp = incumplen.groupby(by="CODIGO_CLIENTE").agg(
    SUMA_FACTOR_TASA_CLIENTE=("FACTOR_TASA", "sum"),
    SUMA_MONTO_CLIENTE=("MONTO_DESEMBOLSADO", "sum")
).reset_index()

incumplen_temp.columns = ["CODIGO_CLIENTE", "SUMA_FACTOR_TASA_CLIENTE", "SUMA_MONTO_CLIENTE"]
incumplen = pd.merge(incumplen, incumplen_temp, how="inner", on="CODIGO_CLIENTE")
incumplen["TPP CLIENTE"] = incumplen["SUMA_FACTOR_TASA_CLIENTE"] / incumplen["SUMA_MONTO_CLIENTE"]

## Simulaciones

Acá empezamos las simulaciones. En el bloque anterior, calculamos todo lo que sería cancelado, ahora procedemos a ver qué pasaría al aplicar la iniciativa.

In [10]:
incumplen["MONTO OFERTA"] = incumplen["DEUDA_INTERNA"] + 50000
incumplen["FACTOR BIENESTAR RECREDITO"] = incumplen.apply(lambda x: factor_bienestar(x["MONTO OFERTA"]), axis=1)
incumplen["RESERVA REESTRUCTURA"] = incumplen["MONTO OFERTA"] * 0.45 * 0.086
incumplen["TASA CVF"] = incumplen.apply(lambda x: cvf(x["MF CANCELADO CLIENTE"], x["FACTOR BIENESTAR RECREDITO"], 
                                                      50000, x["RESERVA_CONSTITUIDA"],
                                                      x["RESERVA REESTRUCTURA"], x["MONTO OFERTA"], alpha), axis=1)

Evaluar si el margen financiero es positivo.

In [11]:
incumplen["INGRESOS FINANCIEROS A 12 MESES"] = incumplen.apply(lambda x: ingresos_financieros(x["MONTO OFERTA"],
                                                                                      x["TASA CVF"]/1200, 120, 12), axis=1) 

incumplen["PASIVOS RECREDITO"] = incumplen.apply(lambda x: ingresos_financieros(x["MONTO OFERTA"], alpha/12, 
                                                                                120, 12), axis=1)

incumplen["PORCENTAJE NUEVA COMISION"] = incumplen.apply(lambda x: porcentaje_comision_desembolso(x["MONTO OFERTA"]), axis=1)

incumplen["COMISIONES NUEVAS 12 MESES"] = incumplen.apply(lambda x: ingresos_financieros(x["MONTO OFERTA"], x["PORCENTAJE NUEVA COMISION"]/12, 120, 12), axis=1)
incumplen["COMISIONES NUEVAS FIN DE AÑO"] = incumplen.apply(lambda x: ingresos_financieros(x["MONTO OFERTA"], x["PORCENTAJE NUEVA COMISION"]/12, 120, 12 - no_mes_simulacion), axis=1)


# AHORA vista fin de año
incumplen["INGRESOS FINANCIEROS FIN DE AÑO"] = incumplen.apply(lambda x: ingresos_financieros(x["MONTO OFERTA"],
                                                                                      x["TASA CVF"]/1200, 120, 12 - no_mes_simulacion), axis=1) 

incumplen["PASIVOS RECREDITO FIN DE AÑO"] = incumplen.apply(lambda x: ingresos_financieros(x["MONTO OFERTA"], alpha/12, 
                                                                                120, 12 - no_mes_simulacion), axis=1)



# MF NUEVO

incumplen["MF RECREDITO 12 MESES"] = incumplen["INGRESOS FINANCIEROS A 12 MESES"] - incumplen["PASIVOS RECREDITO"] - incumplen["RESERVA REESTRUCTURA"] + incumplen["COMISIONES NUEVAS 12 MESES"] - 0.008 * 50000
incumplen["MF RECREDITO FIN DE AÑO"] = incumplen["INGRESOS FINANCIEROS FIN DE AÑO"] - incumplen["PASIVOS RECREDITO FIN DE AÑO"] - incumplen["RESERVA REESTRUCTURA"] + incumplen["COMISIONES NUEVAS FIN DE AÑO"] - 0.008 * 50000

## Resumen del modelo

In [12]:
# Quedarse con los que cumplne
incumplen["DIFERENCIA DE MARGENES CONTANDO COMISIONES 12 MESES"] = incumplen["MF RECREDITO 12 MESES"] - incumplen["MF CANCELADO CLIENTE"] + incumplen["COMISIONES QUE SE TRAERIAN SI SE HACE RECREDITO"]
incumplen["DIFERENCIA DE MARGENES FIN DE AÑO"] = incumplen["MF RECREDITO FIN DE AÑO"] - incumplen["MF A CANCELAR FIN DE AÑO"] + incumplen["COMISIONES QUE SE TRAERIAN SI SE HACE RECREDITO"]
clientes_viables = incumplen.drop_duplicates(subset="CODIGO_CLIENTE")
clientes_viables = incumplen[incumplen["DIFERENCIA DE MARGENES CONTANDO COMISIONES 12 MESES"] <= 0]
tpp_cancelada = sum(clientes_viables["TPP CLIENTE"] * clientes_viables["DEUDA_INTERNA"]) / sum(clientes_viables["DEUDA_INTERNA"])
tpp_recredito = sum(clientes_viables["TASA CVF"] * clientes_viables["MONTO OFERTA"]) / sum(clientes_viables["MONTO OFERTA"])

In [13]:
clientes_viables["DESEMBOLSO NETO NUEVO"] = 50000

In [14]:
# resumir
dic_impactos = {"CANTIDAD DE CLIENTES A ATENDER: ": [len(clientes_viables)],
                "MF NUEVO A 12 MESES EN MILLONES: ": [clientes_viables["MF RECREDITO 12 MESES"].sum() / 10**6],
                "MF FIN DE AÑO EN MILLONES: ": [clientes_viables["MF RECREDITO FIN DE AÑO"].sum() / 10**6],
                "RESERVAS REESTRUCTURA EN MILLONES: ": [clientes_viables["RESERVA REESTRUCTURA"].sum() / 10**6],
                "INGRESOS A 12 MESES EN MILLONES: ": [clientes_viables["INGRESOS FINANCIEROS A 12 MESES"].sum() / 10**6],
                "INGRESOS A FIN DE AÑO EN MILLONES: ": [clientes_viables["INGRESOS FINANCIEROS FIN DE AÑO"].sum() / 10**6],
                "COMISIONES A TRAERSE EN MILLONES: ": [clientes_viables["COMISIONES_A_TRAERSE"].sum() / 10**6],
                "COMISIONES NUEVAS FIN DE AÑO EN MILLONES": [clientes_viables["COMISIONES NUEVAS FIN DE AÑO"].sum() / 10**6],
                "COMISIONES NUEVAS 12 MESES EN MILLONES: ": [clientes_viables["COMISIONES NUEVAS 12 MESES"].sum() / 10**6],
                "PROPORCION CLIENTES CON TPP A LO SUMO 17: ": [len(clientes_viables[clientes_viables["TPP CLIENTE"] <= 17])/len(clientes_viables)],
                "DIFERENCIA DE MARGEN FIN DE AÑO EN MILLONES: ": [clientes_viables["DIFERENCIA DE MARGENES FIN DE AÑO"].sum() / 10**6],
                "DIFERENCIA DE MARGEN 12 MESES EN MILLONES: ": [clientes_viables["DIFERENCIA DE MARGENES CONTANDO COMISIONES 12 MESES"].sum() / 10**6],
                "POTENCIAL DE DESEMBOLSO BRUTO EN MILLONES:": [clientes_viables["MONTO OFERTA"].sum() / 10**6],
                "POTENCIAL DE DESEMBOLSO NETO EN MILLONES: ": [clientes_viables["DESEMBOLSO NETO NUEVO"].sum() / 10**6],
                "TPP CANCELADA: ": [tpp_cancelada],
                "TPP NUEVA RECREDITO: ": [tpp_recredito],
                "PROMEDIO DE SALDO CANCELADO: ": [clientes_viables["DEUDA_INTERNA"].mean()],
                "PROMEDIO DE DESEMBOLSO BRUTO: ": [clientes_viables["MONTO OFERTA"].mean()],
                "PROMEDIO DE DESEMBOLSO NETO:": [50000]}

In [ ]:
df_impactos = pd.DataFrame.from_dict(dic_impactos)
df_impactos.transpose()

In [16]:
clientes_viables["APTO PARA FIN DE AÑO"] = np.where(clientes_viables["MF RECREDITO FIN DE AÑO"] >= 0, 1, 0)
cols = pd.read_excel("../db/Columnas.xlsx").columns.tolist()
clientes_viables = clientes_viables[cols]
clientes_viables.to_csv("../db/Viables.csv")

In [ ]:
cartera[cartera["DESEMBOLSO_NETO"] != cartera["MONTO_DESEMBOLSADO"]]